In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import json
import csv
import sys
import time

ckey = 'bXRPnBdpsgRdrkfbrHBHFInjl'
csecret = 'xm93rCpGvtV6E2lwx6392l5uUfYpz9dgCe7zREGtCWeWd5b6WB'
atoken = '1191115298788777987-qUAhRZ5dAw2wN1QewZxjROvtnVLHBs'
asecret = '595HhAG8XXlbN7BPYfLVUio1ozLqmLMnFDDIaHfzPmeSv'

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets]
    DataSet['Language'] = [tweet.user.lang for tweet in tweets]
    tweets_place= []
    #users_retweeted = []
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    DataSet['TweetPlace'] = [i for i in tweets_place]
    #DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]

    return DataSet

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#auth = tweepy.AppAuthHandler('key', 'key')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
else:
# I am trying to download from Dec 1st to Dec 7th but I am not able to

    cursor = tweepy.Cursor(api.search, q='#dumptrump2020',lang='en')
    results=[]
    for item in cursor.items(10000):
        results.append(item)

    DataSet = toDataFrame(results)

Rate limit reached. Sleeping for: 492


In [2]:
DataSet["text"] = DataSet["tweetText"]
DataSet.shape

(10000, 20)

In [3]:
import re
import preprocessor as p

DataSet["text"] = DataSet["tweetText"]

pd.options.display.max_colwidth = 500

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

DataSet["text"]  = DataSet.apply(preprocess_tweet, axis=1)

DataSet["text"] = DataSet["text"].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


DataSet[["tweetText", "text"]].head()

,tweetText,text
0,"😳Seriously 😳. Give them all the kool aid, except @POTUS Give him the bleach cocktail. #DumpTrump2020… https://t.co/5SF8uc9C35",seriously give them all the kool aid except give him the bleach cocktail
1,@brithume That is a man who takes this virus seriously. He wears mask to protect others. Trump is holding super-spr… https://t.co/o8qoCChkss,that is a man who takes this virus seriously he wears mask to protect others trump is holding super spr
2,RT @Theguyster: @SteveSchmidtSES @TheRickWilson Go Steve! Keep it coming! #DumpTrump2020,go steve keep it coming
3,"@JimboAlogo61 Happy birthday, James! May you receive your best birthday gift a month from today! #DumpTrump2020",happy birthday james may you receive your best birthday gift a month from today
4,@AmyKremer Lol!!! Go Biden!!! Trump was dumb enough to not follow his own administrations guidelines.... it’s like… https://t.co/9VoasBpWeK,lol go biden trump was dumb enough to not follow his own administrations guidelines its like


In [4]:
from nltk.tokenize import word_tokenize



def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
DataSet['text2'] = DataSet['text'].apply(custom_tokenize)


DataSet[["text", "text2"]].head()


The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None type. Defaulting to blank string.
The text to be tokenized is a None

,text,text2
0,seriously give them all the kool aid except give him the bleach cocktail,"[seriously, give, them, all, the, kool, aid, except, give, him, the, bleach, cocktail]"
1,that is a man who takes this virus seriously he wears mask to protect others trump is holding super spr,"[that, is, a, man, who, takes, this, virus, seriously, he, wears, mask, to, protect, others, trump, is, holding, super, spr]"
2,go steve keep it coming,"[go, steve, keep, it, coming]"
3,happy birthday james may you receive your best birthday gift a month from today,"[happy, birthday, james, may, you, receive, your, best, birthday, gift, a, month, from, today]"
4,lol go biden trump was dumb enough to not follow his own administrations guidelines its like,"[lol, go, biden, trump, was, dumb, enough, to, not, follow, his, own, administrations, guidelines, its, like]"


In [5]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

DataSet['text3'] = DataSet['text2'].apply(lambda x: [stemmer.stem(y) for y in x]) 


DataSet[["tweetText", "text2", "text3"]].head()

,tweetText,text2,text3
0,"😳Seriously 😳. Give them all the kool aid, except @POTUS Give him the bleach cocktail. #DumpTrump2020… https://t.co/5SF8uc9C35","[seriously, give, them, all, the, kool, aid, except, give, him, the, bleach, cocktail]","[serious, give, them, all, the, kool, aid, except, give, him, the, bleach, cocktail]"
1,@brithume That is a man who takes this virus seriously. He wears mask to protect others. Trump is holding super-spr… https://t.co/o8qoCChkss,"[that, is, a, man, who, takes, this, virus, seriously, he, wears, mask, to, protect, others, trump, is, holding, super, spr]","[that, is, a, man, who, take, this, virus, serious, he, wear, mask, to, protect, other, trump, is, hold, super, spr]"
2,RT @Theguyster: @SteveSchmidtSES @TheRickWilson Go Steve! Keep it coming! #DumpTrump2020,"[go, steve, keep, it, coming]","[go, steve, keep, it, come]"
3,"@JimboAlogo61 Happy birthday, James! May you receive your best birthday gift a month from today! #DumpTrump2020","[happy, birthday, james, may, you, receive, your, best, birthday, gift, a, month, from, today]","[happi, birthday, jame, may, you, receiv, your, best, birthday, gift, a, month, from, today]"
4,@AmyKremer Lol!!! Go Biden!!! Trump was dumb enough to not follow his own administrations guidelines.... it’s like… https://t.co/9VoasBpWeK,"[lol, go, biden, trump, was, dumb, enough, to, not, follow, his, own, administrations, guidelines, its, like]","[lol, go, biden, trump, was, dumb, enough, to, not, follow, his, own, administr, guidelin, it, like]"


In [6]:
import datetime
year = datetime.date.today().year
month = datetime.date.today().month
month = '{:02d}'.format(month)
day = datetime.date.today().day
day = '{:02d}'.format(day)

today = datetime.date.today().strftime("%Y.%m.%d")
DataSet["keresoszo"] = "#dumptrump2020"
DataSet["letoltes_datum"] = today

In [7]:
DataSet.to_csv('data/dumptrump2020' + str(year) + str(month) + str(day) + '.csv',index=False)

In [8]:
DataSet.head()

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,...,userTimezone,Coordinates,GeoEnabled,Language,TweetPlace,text,text2,text3,keresoszo,letoltes_datum
0,1312408565341134848,"😳Seriously 😳. Give them all the kool aid, except @POTUS Give him the bleach cocktail. #DumpTrump2020… https://t.co/5SF8uc9C35",0,0,Twitter for iPhone,2020-10-03 15:06:00,68557374,groft7,bruce groft,2009-08-25 00:29:27,...,None,None,True,None,"Pennsylvania, USA",seriously give them all the kool aid except give him the bleach cocktail,"[seriously, give, them, all, the, kool, aid, except, give, him, the, bleach, cocktail]","[serious, give, them, all, the, kool, aid, except, give, him, the, bleach, cocktail]",#dumptrump2020,2020.10.03
1,1312408532785025031,@brithume That is a man who takes this virus seriously. He wears mask to protect others. Trump is holding super-spr… https://t.co/o8qoCChkss,0,0,Twitter for iPad,2020-10-03 15:05:52,854563973773045761,kmohairsweater,KMurrayMohairSweater,2017-04-19 05:15:03,...,None,None,False,None,null,that is a man who takes this virus seriously he wears mask to protect others trump is holding super spr,"[that, is, a, man, who, takes, this, virus, seriously, he, wears, mask, to, protect, others, trump, is, holding, super, spr]","[that, is, a, man, who, take, this, virus, serious, he, wear, mask, to, protect, other, trump, is, hold, super, spr]",#dumptrump2020,2020.10.03
2,1312408470054821890,RT @Theguyster: @SteveSchmidtSES @TheRickWilson Go Steve! Keep it coming! #DumpTrump2020,3,0,Twitter for iPhone,2020-10-03 15:05:37,1166730461176946690,DavidBa37028273,David Barr,2019-08-28 15:13:34,...,None,None,False,None,null,go steve keep it coming,"[go, steve, keep, it, coming]","[go, steve, keep, it, come]",#dumptrump2020,2020.10.03
3,1312408206237368320,"@JimboAlogo61 Happy birthday, James! May you receive your best birthday gift a month from today! #DumpTrump2020",0,1,Twitter Web App,2020-10-03 15:04:34,911748369286037504,PitDaddy30701,David Edwards (#BidenHarris2020 #DumpTrump #FBR),2017-09-24 00:25:25,...,None,None,False,None,null,happy birthday james may you receive your best birthday gift a month from today,"[happy, birthday, james, may, you, receive, your, best, birthday, gift, a, month, from, today]","[happi, birthday, jame, may, you, receiv, your, best, birthday, gift, a, month, from, today]",#dumptrump2020,2020.10.03
4,1312408093616041984,@AmyKremer Lol!!! Go Biden!!! Trump was dumb enough to not follow his own administrations guidelines.... it’s like… https://t.co/9VoasBpWeK,0,0,Twitter for iPhone,2020-10-03 15:04:07,813931796765216768,RNA2021,Renee,2016-12-28 02:17:17,...,None,None,False,None,null,lol go biden trump was dumb enough to not follow his own administrations guidelines its like,"[lol, go, biden, trump, was, dumb, enough, to, not, follow, his, own, administrations, guidelines, its, like]","[lol, go, biden, trump, was, dumb, enough, to, not, follow, his, own, administr, guidelin, it, like]",#dumptrump2020,2020.10.03
